In [1]:
import pandas as pd
import duckdb

In [2]:
barcodes = pd.read_csv('child.csv')
products = pd.read_csv('PRODUCTS.csv')

In [3]:
# products.columns

In [4]:
# products.drop(['PRODUCT SKU DESCRIPTION', 'TAX CODE', 'PURCHASING UNIT', 'PRODUCT  COMMENT', 'PRODUCT  BV ID', 'PRODUCT USER ID', 'PRODUCT MANUFACTURER ID'], axis=1).head()
products = products[['VENDOR NAMES', 'CATEGORY NAMES', 'PRODUCT NAME', 'PRODUCT SKU DEPARTMENT', 'PRODUCT TRADE PRICE', 'PRODUCT MAXIMUM RETAIL PRICE',
'PRODUCT BARCODE', 'PRODUCT ID']]

In [5]:
barcodes = barcodes[['inward_child', 'product_id', 'second_level', 'third_level', 'inward_id', 'is_received', 'pick_list_id', 'loc_name', 'location_id']]

In [6]:
# Assuming 'products' is your pandas DataFrame
products.columns = products.columns.str.lower().str.replace(' ', '_')

In [7]:
barcodes.rename(columns={
    'pick_list_id': 'is_sold',
    'loc_name': 'location_name'
},
inplace=True)

In [8]:
barcodes['is_sold'] = duckdb.query(
    """
    SELECT CASE 
           WHEN is_sold IS NOT NULL THEN 'YES' 
           ELSE 'NO' 
       END AS is_sold 
FROM barcodes

    """
).to_df()

In [9]:
products.head()

,vendor_names,category_names,product_name,product_sku_department,product_trade_price,product_maximum_retail_price,product_barcode,product_id
0,A&T ENTERPRISES;RP Legacy Stock,VAGINAL CARE,VAGICARE TABLET 6S,MEDICINES,510.000000,600.0,NaN,1000
1,A.A. TRADERS;PHARMA NET;RP Legacy Stock,DYSPEPESIA,GANATON 50MG TABLET 3X10S,MEDICINES,761.179993,995.0,NaN,1001
2,A.A. TRADERS;RP Legacy Stock;UDL DISTRIBUTION ...,BACTERIAL INFECTION,MOKSI TABLET 400 MG 5S,MEDICINES,663.000000,780.0,NaN,1002
3,A.A. TRADERS;CASH PURCHASES;HASHMANIS (PVT)LTD...,VERTIGO,SERC TABLET 16 MG 3X10S,MEDICINES,510.000000,805.0,NaN,1003
4,A.A. TRADERS;DHA Legacy Stock;HASHMANIS (PVT)L...,VERTIGO,SERC TABLET 8 MG 3X10S,MEDICINES,330.000000,485.0,NaN,1004


In [10]:
barcodes.rename(
    columns={
        'inward_child': 'barcode',
        'inward_id': 'batch_id'
    },
    inplace=True
)

In [11]:
barcodes.head()

,barcode,product_id,second_level,third_level,batch_id,is_received,is_sold,location_name,location_id
0,5732~AAA0001~42,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
1,5732~AAA0001~44,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
2,5732~AAA0001~45,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
3,5732~AAA0001~46,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
4,5732~AAA0001~47,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1


In [18]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

def create_database_if_not_exists(db_params, database_name):
    """
    Creates a PostgreSQL database if it doesn't exist
    """
    # Create a connection to PostgreSQL server (not a specific database)
    conn = psycopg2.connect(
        host=db_params['host'],
        port=db_params['port'],
        user=db_params['user'],
        password=db_params['password'],
        database='mydb'
    )
    
    # Set isolation level to AUTOCOMMIT
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    
    # Create a cursor object
    cur = conn.cursor()
    
    try:
        # Check if database exists
        cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s", (database_name,))
        exists = cur.fetchone()
        
        if not exists:
            # Create the database
            cur.execute(f'CREATE DATABASE {database_name}')
            print(f"Database {database_name} created successfully!")
        else:
            print(f"Database {database_name} already exists!")
            
    except Exception as e:
        print(f"An error occurred: {e}")
        raise
    finally:
        # Close cursor and connection
        cur.close()
        conn.close()

def upload_data_to_postgres(db_params, barcodes_df, products_df):
    """
    Uploads DataFrames to PostgreSQL database
    """
    try:
        # Create SQLAlchemy engine
        engine = create_engine(
            f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@'
            f'{db_params["host"]}:{db_params["port"]}/{db_params["database"]}'
        )
        
        # Upload DataFrames
        barcodes_df.to_sql('barcodes', engine, index=False, if_exists='replace')
        products_df.to_sql('products', engine, index=False, if_exists='replace')
        
        print("Data uploaded successfully!")
        
    except Exception as e:
        print(f"An error occurred while uploading data: {e}")
        raise
    finally:
        engine.dispose()

# Define connection parameters
db_params = {
    'host': 'localhost',
    'port': '5432',
    'user': 'admin',
    'password': 'admin'
}

# Database name
database_name = 'ninja_db'

# Create database
create_database_if_not_exists(db_params, database_name)

# Update db_params with database name
db_params['database'] = database_name

# Assuming barcodes and products DataFrames exist
# upload_data_to_postgres(db_params, barcodes, products)

Database ninja_db created successfully!


In [20]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

def upload_dataframes_to_postgres(db_params, barcodes_df, products_df):
    """
    Upload Pandas DataFrames to PostgreSQL tables
    """
    try:
        # Create the connection string
        engine = create_engine(
            f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@'
            f'{db_params["host"]}:{db_params["port"]}/{db_params["database"]}'
        )
        
        print("Starting data upload...")
        
        # Upload barcodes DataFrame
        print("Uploading barcodes table...")
        barcodes_df.to_sql(
            'barcodes', 
            engine, 
            index=False, 
            if_exists='replace',
            chunksize=1000  # Upload in chunks to handle large datasets
        )
        print(f"✓ Barcodes table uploaded successfully ({len(barcodes_df)} rows)")
        
        # Upload products DataFrame
        print("Uploading products table...")
        products_df.to_sql(
            'products', 
            engine, 
            index=False, 
            if_exists='replace',
            chunksize=1000
        )
        print(f"✓ Products table uploaded successfully ({len(products_df)} rows)")
        
        # Verify the upload
        with engine.connect() as conn:
            # Check barcodes table
            result = conn.execute("SELECT COUNT(*) FROM barcodes")
            barcodes_count = result.scalar()
            
            # Check products table
            result = conn.execute("SELECT COUNT(*) FROM products")
            products_count = result.scalar()
            
        print("\nVerification Results:")
        print(f"Barcodes table row count: {barcodes_count}")
        print(f"Products table row count: {products_count}")
        
        return True
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return False
        
    finally:
        engine.dispose()

# Database connection parameters
db_params = {
    'host': 'localhost',
    'port': '5432',
    'user': 'admin',
    'password': 'admin',
    'database': 'mydb'
}

# Upload the DataFrames
success = upload_dataframes_to_postgres(db_params, barcodes, products)

if success:
    print("\nData upload completed successfully!")
else:
    print("\nData upload failed. Please check the error messages above.")

Starting data upload...
Uploading barcodes table...
✓ Barcodes table uploaded successfully (2774001 rows)
Uploading products table...
✓ Products table uploaded successfully (7451 rows)
Error occurred: Not an executable object: 'SELECT COUNT(*) FROM barcodes'

Data upload failed. Please check the error messages above.


In [21]:
barcodes.head()

,barcode,product_id,second_level,third_level,batch_id,is_received,is_sold,location_name,location_id
0,5732~AAA0001~42,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
1,5732~AAA0001~44,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
2,5732~AAA0001~45,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
3,5732~AAA0001~46,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1
4,5732~AAA0001~47,5732,1,1.0,5732~AAA0001,1,NO,WAREHOUSE,1


In [22]:
duckdb.query(
    """
    select * from products
    where product_id = 5432
    """
)

┌─────────────────────────────────────────────────────────┬────────────────┬─────────────────────────────┬────────────────────────┬─────────────────────┬──────────────────────────────┬─────────────────┬────────────┐
│                      vendor_names                       │ category_names │        product_name         │ product_sku_department │ product_trade_price │ product_maximum_retail_price │ product_barcode │ product_id │
│                         varchar                         │    varchar     │           varchar           │        varchar         │       double        │            double            │     double      │   int64    │
├─────────────────────────────────────────────────────────┼────────────────┼─────────────────────────────┼────────────────────────┼─────────────────────┼──────────────────────────────┼─────────────────┼────────────┤
│ CASH PURCHASES;RP Legacy Stock;UDL DISTRIBUTION PVT LTD │ EYE ALLERGY    │ XEPAT FORTE HCL 0.2% 5ML 1S │ MEDICINES              │   49